<a href="https://colab.research.google.com/github/TheSupremeTaco/Ancestry_Linear_Reg/blob/main/DeepLearning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>